In [1]:
from importlib import reload
import sys
import os
sys.path.insert(0, './../scripts')

In [2]:
import h5py
import pickle
import numpy as np
import torch
import torch_interpolations as torchitp
from torchquad import Simpson, set_up_backend
from scipy.interpolate import RegularGridInterpolator
from scipy.integrate import quad
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.figure import figaspect
import time

import ring_network as network
import sim_util as su
import ricciardi as ric
import integrate as integ

def imshowbar(fig,ax,A,**kwargs):
    imsh = ax.imshow(A,**kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(imsh, cax=cax, orientation='vertical')

# Read itp.h5, Interpolate Base Moments

In [4]:
def μtox(μ):
    return np.sign(μ/100-0.2)*np.abs(μ/100-0.2)**0.5

def xtoμ(x):
    return 100*(np.sign(x)*np.abs(x)**2.0+0.2)

def extract_itp(file):
    f = h5py.File(file, 'r')
    
    φxrange = list(f['PhItp']['xrange'])
    φxs = np.linspace(φxrange[0],φxrange[1],round(φxrange[2]))
    φEs = np.array(f['PhItp']['rp1']['Phs'])
    φIs = np.array(f['PhItp']['rp2']['Phs'])
    
    Mxrange = list(f['MItp']['xrange'])
    Mxs = np.linspace(Mxrange[0],Mxrange[1],round(Mxrange[2]))
    Mσrange = list(f['MItp']['srange'])
    Mσs = np.linspace(Mσrange[0],Mσrange[1],round(Mσrange[2]))
    MEs = np.array(f['MItp']['rp1']['Ms'])
    MIs = np.array(f['MItp']['rp2']['Ms'])
    
    Cxrange = list(f['CItp']['xrange'])
    Cxs = np.linspace(Cxrange[0],Cxrange[1],round(Cxrange[2]))
    Cσrange = list(f['CItp']['srange'])
    Cσs = np.linspace(Cσrange[0],Cσrange[1],round(Cσrange[2]))
    Ccrange = list(f['CItp']['crange'])
    Ccs = np.linspace(Ccrange[0],Ccrange[1],round(Ccrange[2]))
    CEs = np.array(f['CItp']['rp1']['Cs'])
    CIs = np.array(f['CItp']['rp2']['Cs'])
    
    φE_itp = RegularGridInterpolator((φxs,),φEs,bounds_error=False,fill_value=None)
    φI_itp = RegularGridInterpolator((φxs,),φIs,bounds_error=False,fill_value=None)
    
    ME_itp = RegularGridInterpolator((Mσs,Mxs),MEs,bounds_error=False,fill_value=None)
    MI_itp = RegularGridInterpolator((Mσs,Mxs),MIs,bounds_error=False,fill_value=None)
    
    CE_itp = RegularGridInterpolator((Ccs,Cσs,Cxs),CEs,bounds_error=False,fill_value=None)
    CI_itp = RegularGridInterpolator((Ccs,Cσs,Cxs),CIs,bounds_error=False,fill_value=None)
    
    base_itp_dict = {}
    base_itp_dict['φE_itp'] = φE_itp
    base_itp_dict['φI_itp'] = φI_itp
    base_itp_dict['ME_itp'] = ME_itp
    base_itp_dict['MI_itp'] = MI_itp
    base_itp_dict['CE_itp'] = CE_itp
    base_itp_dict['CI_itp'] = CI_itp
    
    with open('base_itp'+'.pkl', 'wb') as handle:
        pickle.dump(base_itp_dict,handle)
    
    def φE(μ):
        try:
            return φE_itp(μtox(1e3*μ)[:,None])
        except:
            return φE_itp([μtox(1e3*μ)])
    def φI(μ):
        try:
            return φI_itp(μtox(1e3*μ)[:,None])
        except:
            return φI_itp([μtox(1e3*μ)])
    
    def ME(μ,Σ):
        return ME_itp(np.vstack((1e3*np.sqrt(Σ),μtox(1e3*μ))).T)
    def MI(μ,Σ):
        return MI_itp(np.vstack((1e3*np.sqrt(Σ),μtox(1e3*μ))).T)
    
    def CE(μ,Σ,k):
        c = np.sign(k)*np.fmin(np.abs(k)/Σ,1)
        return CE_itp(np.vstack((c,1e3*np.sqrt(Σ),μtox(1e3*μ))).T)
    def CI(μ,Σ,k):
        c = np.sign(k)*np.fmin(np.abs(k)/Σ,1)
        return CI_itp(np.vstack((c,1e3*np.sqrt(Σ),μtox(1e3*μ))).T)
    
    return φE,φI,ME,MI,CE,CI

In [5]:
φE,φI,ME,MI,CE,CI = extract_itp('itp.h5')

# Define Base Ring DMFT Functions

In [ ]:
function dmft()